## Task1
Reimplement the feed-forward neural network from Week 2 in PyTorch, using
the same subset of the AGnews dataset – this time we have not provided any starting
point for you, so you need to create a new notebook for this. This notebook is one of
two notebooks that you need to submit for Week 3.

In [50]:
import torch
from torch import nn

from torchvision import datasets
from torchvision.transforms import ToTensor

In [51]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


## Define the Class
We define our neural network by subclassing ``nn.Module``, and
initialize the neural network layers in ``__init__``. Every ``nn.Module`` subclass implements
the operations on input data in the ``forward`` method.

In [52]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(1500, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 3),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of ``NeuralNetwork``, and move it to the ``device``, and print
its structure.

In [53]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=1500, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=3, bias=True)
  )
)


## Get Training and Test Data

In [54]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.feature_extraction.text import TfidfVectorizer

def preprocess_agnews(csv_file):
    '''Load and preprocess AG News data'''
    df = pd.read_csv(csv_file, header = None)
    df = df[df[0] !=1]
    df['label'] = df[0] - 2
    df['text'] = [" ".join((title, body)) for title, body in zip(df[1], df[2])]
    return df[['text', 'label']]

class TextDataset(Dataset):
    def __init__(self, dataframe, max_features=1500, vectoriser=None):
        self.texts = dataframe['text'].tolist()
        self.labels = dataframe['label'].tolist()
        
        # TF-IDF: fit on training, reuse for testing
        if vectoriser is None:
            # Training: create and fit
            self.vectoriser = TfidfVectorizer(max_features=max_features, stop_words='english')
            self.features = self.vectoriser.fit_transform(self.texts)
        else:
            # Testing: use provided vectoriser
            self.vectoriser = vectoriser
            self.features = self.vectoriser.transform(self.texts)
        
        # Convert sparse matrix to dense tensor (TF-IDF returns sparse)
        self.features = torch.tensor(self.features.toarray(), dtype=torch.float32)
        self.labels = torch.tensor(self.labels, dtype=torch.long)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        # Just return pre-computed vectors
        return self.features[idx], self.labels[idx]

In [55]:
train_df = preprocess_agnews('../W2_feedfoward_neural_networks/agnews_train.csv')
test_df = preprocess_agnews('../W2_feedfoward_neural_networks/agnews_test.csv')

train_dataset = TextDataset(train_df, max_features=1500)
test_dataset = TextDataset(test_df, vectoriser=train_dataset.vectoriser)  # Passing the fitted vectoriser

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [56]:
model = NeuralNetwork()

## Hyperparameters

Hyperparameters are adjustable parameters that let you control the model optimization process.
Different hyperparameter values can impact model training and convergence rates
([read more](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)_ about hyperparameter tuning)

We define the following hyperparameters for training:
 - **Number of Epochs** - the number times to iterate over the dataset
 - **Batch Size** - the number of data samples propagated through the network before the parameters are updated
 - **Learning Rate** - how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behavior during training.




In [57]:
learning_rate = 0.01
batch_size = 64
epochs = 5

## Optimization Loop

Once we set our hyperparameters, we can then train and optimize our model with an optimization loop. Each
iteration of the optimization loop is called an **epoch**.

Each epoch consists of two main parts:
 - **The Train Loop** - iterate over the training dataset and try to converge to optimal parameters.
 - **The Validation/Test Loop** - iterate over the test dataset to check if model performance is improving.

Let's briefly familiarize ourselves with some of the concepts used in the training loop. Jump ahead to
see the `full-impl-label` of the optimization loop.

### Loss Function

When presented with some training data, our untrained network is likely not to give the correct
answer. **Loss function** measures the degree of dissimilarity of obtained result to the target value,
and it is the loss function that we want to minimize during training. To calculate the loss we make a
prediction using the inputs of our given data sample and compare it against the true data label value.

Common loss functions include [nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss) (Mean Square Error) for regression tasks, and
[nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss) (Negative Log Likelihood) for classification.
[nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) combines ``nn.LogSoftmax`` and ``nn.NLLLoss``.

We pass our model's output logits to ``nn.CrossEntropyLoss``, which will normalize the logits and compute the prediction error.



In [58]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### Optimizer

Optimization is the process of adjusting model parameters to reduce model error in each training step. **Optimization algorithms** define how this process is performed (in this example we use Stochastic Gradient Descent).
All optimization logic is encapsulated in  the ``optimizer`` object. Here, we use the SGD optimizer; additionally, there are many [different optimizers](https://pytorch.org/docs/stable/optim.html)
available in PyTorch such as ADAM and RMSProp, that work better for different kinds of models and data.

We initialize the optimizer by registering the model's parameters that need to be trained, and passing in the learning rate hyperparameter.

In [59]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Implementing the model

In [60]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

We initialize the loss function and optimizer, and pass it to ``train_loop`` and ``test_loop``.
Feel free to increase the number of epochs to track the model's improving performance.

In [ ]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.099327  [   64/90000]
loss: 1.097534  [ 6464/90000]
loss: 1.098192  [12864/90000]
loss: 1.097020  [19264/90000]
loss: 1.098091  [25664/90000]
loss: 1.097373  [32064/90000]
loss: 1.098455  [38464/90000]
loss: 1.097234  [44864/90000]
loss: 1.097577  [51264/90000]
loss: 1.096474  [57664/90000]
loss: 1.098327  [64064/90000]
loss: 1.098231  [70464/90000]
loss: 1.097269  [76864/90000]
loss: 1.097273  [83264/90000]
loss: 1.097051  [89664/90000]
Test Error: 
 Accuracy: 39.8%, Avg loss: 1.096850 

Epoch 2
-------------------------------
loss: 1.097885  [   64/90000]
loss: 1.098149  [ 6464/90000]
loss: 1.097857  [12864/90000]
loss: 1.096896  [19264/90000]
loss: 1.096806  [25664/90000]
loss: 1.096406  [32064/90000]
loss: 1.095656  [38464/90000]
loss: 1.097645  [44864/90000]
loss: 1.095282  [51264/90000]
loss: 1.094797  [57664/90000]
loss: 1.094629  [64064/90000]
loss: 1.094089  [70464/90000]
loss: 1.095634  [76864/90000]
loss: 1.095041  [83264/90000